In [ ]:
!which julia

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" 
JULIA_PACKAGES=""
JULIA_PACKAGES_IF_GPU="CUDA" 
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done
fi

In [ ]:
!which julia

In [ ]:
!pip install pysr

In [ ]:
import pysr

In [ ]:
pysr.install()

In [ ]:
import numpy as np
X = np.random.randn(100, 5)
y = 2.0 * np.cos(X[:, 3]) + X[:,0]**2 - 2.0

In [ ]:
from pysr import PySRRegressor
model = PySRRegressor(
    niterations=5,
    binary_operators=["+", "*", "-", "^", "/"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "tan",
        "inv(x) = 1/x",  # Custom operator (julia syntax)
    ],
    model_selection="best",
    loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
)

In [ ]:
model.fit(X, y)

PySRRegressor.equations = [
	   pick     score                                           equation  \
	0        0.000000                                         0.43449417   
	1        0.150728                                            cos(x3)   
	2        0.816824                                 pow(x0, 1.8281212)   
	3        0.078394                          pow(0.104642525, cos(x0))   
	4        0.384409                   (pow(x0, 1.9785022) - 0.6698592)   
	5        1.729187          (pow(x0, 2.0101287) - pow(x3, 1.3389946))   
	6        0.113102  (pow(x0, 1.9957054) - (pow(x3, 1.347889) + -0....   
	7  >>>>       inf  (((x0 * x0) - (sin(x3 - 1.5707995) * 2.0000012...   
	
	       loss  complexity  
	0  2.885226           1  
	1  2.481528           2  
	2  1.096421           3  
	3  1.013751           4  
	4  0.690215           5  
	5  0.021729           7  
	6  0.017330           9  
	7  0.000000          12  
]

In [ ]:
print(model.latex())

x_{0}^{2} - 2.0000012 \sin{\left(x_{3} - 1.5707995 \right)} - 2.0000012
